In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import shutil
from sklearn.model_selection import GroupShuffleSplit

# Making data format for yolo

In [3]:
scv_path = 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/filtered_csv\\final_dfV2.csv' 

df = pd.read_csv(scv_path)

print("shape of csv: ",df.shape[0]) 
df.head()

shape of csv:  2505516


,ImageID,ClassName,ImagePath,center_x,center_y,width,height,class_int
0,0000048549557964,Building,C:\Users\PC\OneDrive\Desktop\cv assign3 data\t...,0.125312,0.464583,0.250625,0.600833,0
1,0000048549557964,Building,C:\Users\PC\OneDrive\Desktop\cv assign3 data\t...,0.291563,0.614583,0.138125,0.125833,0
2,0000048549557964,Building,C:\Users\PC\OneDrive\Desktop\cv assign3 data\t...,0.564375,0.617916,0.043750,0.070833,0
3,0000048549557964,Building,C:\Users\PC\OneDrive\Desktop\cv assign3 data\t...,0.805312,0.559167,0.388125,0.370000,0
4,0001c6c99f4a838a,Building,C:\Users\PC\OneDrive\Desktop\cv assign3 data\t...,0.499688,0.507974,0.999375,0.982176,0


### Splitting the dataset

In [8]:
#this methid does not work because multiple instance of same image id 
#train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
#print("train data size: ",len(train_df))
#print("val data size: ",len(val_df))

train data size:  2004412
val data size:  501104


In [5]:
#a GroupShuffleSplit instance
splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)

# Splitting the dataframe based on groups formed by 'ImageID'
train_inds, val_inds = next(splitter.split(df, groups=df['ImageID']))


train_df = df.iloc[train_inds]
val_df = df.iloc[val_inds]

print("train data size: ",len(train_df))
print("val data size: ",len(val_df))

E:\Conda\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
E:\Conda\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


train data size:  2005836
val data size:  499680


### saving train and val csv

In [6]:
#train
output_csv_path = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\filtered_csv\\train_df.csv'
train_df.to_csv(output_csv_path, index=False)

#val
output_csv_path = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\filtered_csv\\val_df.csv'
val_df.to_csv(output_csv_path, index=False)

In [2]:
#using saved csv
scv_path = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\filtered_csv\\train_df.csv' 
train_df = pd.read_csv(scv_path)
print("shape of train csv: ",train_df.shape[0])

scv_path = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\filtered_csv\\val_df.csv' 
val_df = pd.read_csv(scv_path)
print("shape of val csv: ",val_df.shape[0])

shape of train csv:  2005836
shape of val csv:  499680


### making annotations for format of yolo

In [7]:
def write_annotations(df, folder_path):
    grouped = df.groupby('ImageID')

    for name, group in grouped:
        filepath = os.path.join(folder_path, name + '.txt')
        with open(filepath, 'w') as file:
            for _, row in group.iterrows():
                class_id = row['class_int']
                bbox = (row['center_x'], row['center_y'], row['width'], row['height'])
                bbox_str = ' '.join([str(x) for x in bbox])
                line = f"{class_id} {bbox_str}\n"
                file.write(line)


In [8]:
write_annotations(train_df, 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/yolo_annotations/train')
write_annotations(val_df, 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/yolo_annotations/val')

## setting up data folder according to yolo standards

### moving main directory images to train and test folders accordingly

In [4]:
#shifting images to train and val folders using their individual dataframes
base_image_dir = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\trainsubset\\'

#directories for train and val datasets
train_dir = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\data\\images\\train\\'
val_dir = 'C:\\Users\\PC\\OneDrive\\Desktop\\cv assign3 data\\data\\images\\val\\'

#chexk for directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

#function to move images based on ImageID from dataframe to their directories
def move_images(df, destination):
    for image_id in df['ImageID']:
        image_name = image_id + '.jpg'  
        source_path = os.path.join(base_image_dir, image_name)
        dest_path = os.path.join(destination, image_name)
        if os.path.exists(source_path):
            shutil.move(source_path, dest_path)


            
move_images(train_df, train_dir)#moving train images
move_images(val_df, val_dir)#moving val images

### making a list of all class names

In [6]:
label_names_csv_path = 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/class-names-boxable.csv'

# Read the CSV file
try:
    label_names_df = pd.read_csv(label_names_csv_path, header=None)
    # Assuming the second column contains the class names
    class_names = label_names_df.iloc[:, 1].tolist()
    # Display the first few class names for verification
    print("First few class names:", class_names[:10])
except Exception as e:
    print("An error occurred while reading the file:", e)

#print(class_names) #prints a lot of classes so good
print(len(class_names))

First few class names: ['Tortoise', 'Container', 'Magpie', 'Sea turtle', 'Football', 'Ambulance', 'Ladder', 'Toothbrush', 'Syringe', 'Sink']
601


### making yaml file

In [7]:
import yaml

data_yaml = {
    'train': 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/data/images/train',
    'val': 'C:/Users/PC/OneDrive/Desktop/cv assign3 data/data/images/val',
    'nc': len(class_names),
    'names': class_names
}

with open('dataset.yaml', 'w') as file:
    yaml.dump(data_yaml, file)

## setting up yolo v5

In [12]:
#already done
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

C:\Users\PC\CV LAB\yolov5


Cloning into 'yolov5'...


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/8d/c4/82b858fb6483dfb5e338123c154d19c043305b01726a67d89532b8f8f01b/GitPython-3.1.40-py3-none-any.whl.metadata
  Obtaining dependency information for ultralytics>=8.0.147 from https://files.pythonhosted.org/packages/35/6b/26ef27d9fac7448d035028b730f4b4f84a382c40a806cda26e077122d6d4/ultralytics-8.0.218-py3-none-any.whl.metadata
  Obtaining dependency information for gitdb<5,>=4.0.1 from https://files.pythonhosted.org/packages/fd/5b/8f0c4a5bb9fd491c277c21eff7ccae71b47d43c4446c9d0c6cff2fe8c2c4/gitdb-4.0.11-py3-none-any.whl.metadata
  Obtaining dependency information for smmap<6,>=3.0.1 from https://files.pythonhosted.org/packages/a7/a5/10f97f73544edcdef54409f1d839f6049a0d79df68adbc1ceb24d1aaca42/smmap-5.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/190.6 kB ? eta -:--:--
   -- ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Running training script - yolov5 

In [13]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.1.0
True


In [12]:
!python yolov5/train.py --img 256 --batch 16 --epochs 50 --data dataset.yaml --weights yolov5s.pt --cache

^C
